In [1]:
import pymysql
import numpy as np
import pandas as pd
import time

MYSQL_CONN = pymysql.connect(
    host='db-6slkr-kr.vpc-pub-cdb.ntruss.com',
    port=3306,
    user='domain',
    passwd='bookcode7!',
    db='bookcode',
    charset='utf8mb4'
)

def conn_mysqldb():
    if not MYSQL_CONN.open:
        MYSQL_CONN.ping(reconnect=True)
    return MYSQL_CONN

if __name__ == '__main__':
    res = conn_mysqldb()
    sql = "SELECT title FROM Book;" 

    cursor = res.cursor() 
    cursor.execute(sql) 

    result = cursor.fetchall()
    result = pd.DataFrame(result)
    
    print(result)
    

                                                      0
0                                        흘러가는 구름을 동경하였다
1                               장애와 테크놀로지 - 과학잡지 에피 16호
2                                  식물의 과학 - 과학잡지 에피 17호
3                          에픽 #04 - BELOVED, 2021.7.8.9
4                               오늘의 좋은 소설 2021.가을 - 58호
...                                                 ...
5901                                          왓더? 오디오 1
5902      오늘의 어린이책 1 - 다움북클럽이 고른 성평등 어린이·청소년책 2019-2021
5903  괜찮아, 아빠도 쉽진 않더라 - 오늘을 살아가는 우리 모두를 위한 동화, &lt;퇴...
5904                                       미래를 향한 노스탤지어
5905          채사장의 지대넓얕 1 : 권력의 탄생 - 지적 대화를 위한 넓고 얕은 지식

[5906 rows x 1 columns]


In [3]:
# 가능성 나열 ㅆㅃ...
# 1. 설득의 심리학 1 (리커버 에디션) - 사람의 마음을 사로잡는 6가지 불변의 법칙   (로split&숫자제거
# 2. 몬테크리스토 백작 3 숫자만 제거
# 4. 오늘의 SF #1 특수문자와 숫자만
# 5. 밤의 대통령 1~2 세트 - 전2권 - 개정판   개정판/세트/한정판 등의 특수단어를 지정하고 replace

import re
text="몬테크리스토 백작 3 "

# 개정판 세트 한정판 등의 특수단어 replace
boollist=['에디션','한정판','세트','리커버','한정']
for boolword in boollist:
    text=text.replace(boolword,'')
print(text)

# 정규식으로 한글과 띄어쓰기만 남기기
letters_only = re.sub('[^ 가-힣]', '', text)   

# ( - : 기준으로 split
letters = text.split('(' or '-' or ':')[0]
best_clean = re.sub('[^ 가-힣]', '', letters)
print(letters_only)
print(letters)
print(text)
print(best_clean)
sysno = [text,letters_only,letters,best_clean]
print(sysno)
sys = [syss.strip() for syss in sysno]
print(list(set(sys)))

몬테크리스토 백작 3 
몬테크리스토 백작  
몬테크리스토 백작 3 
몬테크리스토 백작 3 
몬테크리스토 백작  
['몬테크리스토 백작 3 ', '몬테크리스토 백작  ', '몬테크리스토 백작 3 ', '몬테크리스토 백작  ']
['몬테크리스토 백작', '몬테크리스토 백작 3']


In [84]:
allWordList=[] #중복점검용 유의어까지 모든 단어 저장하는 리스트
synonymList={} #db의 title 이름을 key값으로 title의 유사어를 value 값으로 가지는 딕셔너리

def checkInWordList(wordList):
    for word in wordList:
        if word not in allWordList:
            allWordList.append(word)
        else:
            wordList.remove(word)
    return wordList
     
for title in result[0]:
    title_synonym = []
    if(len(title)>40):
        continue
    # 개정판 세트 한정판 등의 특수단어 replace
    boollist=['에디션','한정판','세트','리커버','한정','특별판','개역판','개정판','재판', '제판','리마스터판','반양장','양장판']
    original_title = title
    for boolword in boollist:
        title=title.replace(boolword,'')
    
    # onlyko_title : 정규식으로 한글과 띄어쓰기만 남기기
    onlyko_title = re.sub('[^ 가-힣]', '', title)   

    # split_title : ( - : 기준으로 split
    split_title= re.sub('[/:(.-]','/',title)
    split_title = split_title.split('/')[0]
    
    # bestclean_title : ( - : 기준으로 split + 정규식으로 한글과 띄어쓰기만
    bestclean_title = re.sub('[^ 가-힣]', '', split_title)
    
    title_synonym.append(onlyko_title)
    title_synonym.append(split_title)
    title_synonym.append(bestclean_title)
    
    # 중복 제거
    title_synonym = list(set(title_synonym))
    title_synonym = [synonym.strip() for synonym in title_synonym]
    try:
        title_synonym.remove(original_title)
    except:
        pass
    title_synonym = checkInWordList(title_synonym)
    
    synonymList[original_title]='/'.join(title_synonym)    

In [92]:
import pandas as pd

# 샘플 데이터프레임 생성
inventors = pd.DataFrame.from_dict([synonymList])
for title in synonymList.keys():
    inventors[title][0]=inventors[title][0].replace('/',',')

In [94]:
# 이제 인덱스는 저장되지 않습니다.
inventors.T.to_excel('bookentity.xlsx', index=True)
#이뒤에 잠깐 파일에 들어가서 임의로 열의 이름을 바꾸고 book을 엔티티 이름 열에 추가해줌

In [95]:
df= pd.read_excel('bookentity.xlsx')

In [96]:
print(df.columns)

Index(['대표어', '유사어', '엔티티 이름', '민감 정보 보안설정'], dtype='object')


In [97]:
df=df.reindex(columns=['엔티티 이름','대표어','유사어','민감 정보 보안설정'] )
df['민감 정보 보안설정']='일반설정'
df.to_excel('bookentity.xlsx', index=False)